# VRP Demo with SCIP
This file is intended to build the model for the CVRP using SCIP

In [1]:
from pyscipopt import Model, Pricer, SCIP_RESULT, SCIP_STAGE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import sys, math, random

from parse import parse

from cffi import FFI
ffi = FFI()
labelling_lib = ffi.dlopen("Labelling/labelling_lib.so")

funDefs = "void initGraph(unsigned num_nodes, unsigned* node_data, double* edge_data, const double capacity, const unsigned max_path_len); unsigned labelling(double const * dual, const bool farkas, const bool elementary, const unsigned long max_vars, const bool cyc2, unsigned* result);"
ffi.cdef(funDefs, override=True)

In [2]:
# Create Graph
G = nx.complete_graph(10)
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = random.randint(1,10)
    
for node in G.nodes():
    G.nodes()[node]['demand'] = random.randint(1,10)

G.graph['capacity'] = 20
# nx.draw(G)

In [3]:
# Create Simple Graph for correctnes
G = nx.complete_graph(4)
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = 1
G.edges[1,2]['weight'] = 1

for node in G.nodes():
    G.nodes()[node]['demand'] = 2

G.graph['capacity'] = 4
# nx.draw(G,with_labels=True)

In [3]:
# Test instance E-n22-k4 provided by parser
G = parse("Instances/E/E-n22-k4.vrp")
# G = parse("Instances/E/E-n23-k3.vrp")
# G = parse("Instances/E/E-n30-k3.vrp")
# G = parse("Instances/E/E-n33-k4.vrp")
# G = parse("Instances/E/E-n51-k5.vrp")
# G = parse("Instances/E/E-n76-k14.vrp")

PARSE: Minimum number of trucks is 4


In [4]:
class VRP(Model):
    def __init__(self,graph):
        super().__init__()
        
        self.original_graph = graph
        self.graph = graph.copy()
        self.vars = {}
        self.cons = []

In [6]:
class VRPPricer(Pricer): 
    def pricerinit(self):
        self.data['cons'] = [self.model.getTransformedCons(con) for con in self.model.cons]
        self.data['vars'] = {path:self.model.getTransformedVar(var) for (path,var) in self.model.vars.items()}
        
#         print(f" There are {len(self.model.getConss())} constraints in the model and {len(self.data['cons'])} of them are known to the pricer.")
        
        demands = list(nx.get_node_attributes(self.model.graph,"demand").values())
        if not np.all(np.array(demands[1:])):
           print("PRICER_PY: The demands of all nodes must be > 0.")
#         print(f"PRICER_PY: The demands are {node_data}")
        nodes_arr = ffi.cast("unsigned*", np.array(demands).astype(np.uintc).ctypes.data)
        
        minimal_demands = sum(sorted(demands[1:])[:2])
        
        self.data['max_path_len'] = math.ceil(2*self.data['capacity'] / minimal_demands) + 2
        print(f"PRICER_PY: The maximal path length is {self.data['max_path_len']}")
    
        edges = nx.adjacency_matrix(self.model.graph,dtype=np.double).toarray()
        edges_arr = ffi.cast("double*", edges.ctypes.data)
        
        num_nodes = ffi.cast("unsigned",self.model.graph.number_of_nodes())
        
        capacity_ptr = ffi.cast("double",self.data['capacity'])
        labelling_lib.initGraph(num_nodes,nodes_arr,edges_arr, capacity_ptr, self.data['max_path_len'])
    
    def pricerfarkas(self):
        dual = [self.model.getDualfarkasLinear(con) for con in self.data['cons']]
#         print(f"PRICER_PY: Farkas Values are {dual}")
        return self.labelling(dual, farkas=True)

    def pricerredcost(self):
        dual = [self.model.getDualsolLinear(con) for con in self.data['cons']]
#         print(f"PRICER_PY: Dual variables are {dual}")
        return self.labelling(dual)
    
    def labelling(self, dual,farkas=False, elementary=False, max_vars=10000, cyc2=False):
        pointer_dual = ffi.cast("double*", np.array(dual,dtype=np.double).ctypes.data)
        
        # TODO: Possible improvement: result can be reused every time

        result = np.zeros(max_vars*self.data['max_path_len'] ,dtype=np.uintc)
        result_arr = ffi.cast("unsigned*",result.ctypes.data)
        
        num_paths = labelling_lib.labelling(pointer_dual, farkas, elementary, max_vars, cyc2, result_arr)
        print(f"PY PRICING: Found {num_paths} paths with reduced cost")
        
        if(num_paths == 0):
            print("PY PRICING: There are no paths with negative reduced costs")
            return {'result':SCIP_RESULT.SUCCESS}
        for i in range(min(num_paths,max_vars)):
            single_result = result[i*self.data['max_path_len']:(i+1)*self.data['max_path_len']]
            result_indices = np.insert(np.nonzero(single_result),0,0)
            result_indices = np.append(result_indices,0)
            path = tuple(single_result[result_indices])
#             print(f"Found path {path}")
            if path in self.data['vars'].keys():
                cost = self.model.getVarRedcost(self.data['vars'][path])
                if farkas:
                    print(f"PY Farkaspricing: Path already exists. | {path}")
                else:
                    print(f"PY Pricing: Path already exists. Reduced Cost {cost:.2f} | {path}")
                return {'result':SCIP_RESULT.SUCCESS}
        
            var = self.model.addVar(vtype="C",obj=nx.path_weight(self.model.graph,path,"weight"),pricedVar=True)
            weight = nx.path_weight(self.model.graph,path,"weight")
        
            counts = np.unique(path[1:-1], return_counts=True)
            for i, node in enumerate(counts[0]):
                self.model.addConsCoeff(self.data['cons'][node-1], var ,counts[1][i])

            self.model.addConsCoeff(self.data['cons'][-1], var, 1)
            self.data['vars'][tuple(path)] = var
        
        return {'result':SCIP_RESULT.SUCCESS}

In [7]:
model = VRP(G)
num_vehicles = G.graph['min_trucks']
print(f"MODEL BUILDER: The number of vehicles is {num_vehicles}")

# Create pricer
pricer = VRPPricer()
pricer.data = {}
pricer.data["capacity"] = G.graph['capacity']
pricer.data["num_vehicles"] = num_vehicles
model.includePricer(pricer, "pricer","does pricing")

# Create a valid set of variables and the constraints to it
for i in range(1,G.number_of_nodes()):
    #TODO: I should check, whether these paths are indeed feasible.
    path = (0,i,0)
    cost = nx.path_weight(G,path,"weight")
#     print(f"Do these costs make sense? {cost}")
    var = model.addVar(vtype="C",obj=cost)
    model.vars[path] = var
    cons = model.addCons(var == 1, name=f"node_{i}",modifiable=True)
    model.cons.append(cons)
    
# Add the convexity constraint, which limits the number of available vehicles
convexity_constraint = model.addCons(sum(model.vars.values()) <= num_vehicles, modifiable=True)
model.cons.append(convexity_constraint)

# model.hideOutput()
model.optimize()
model.hideOutput(quiet=False)
# model.printBestSol()
sol = model.getBestSol()

# Flushing should probably prevent the console output from SCIP mix up with the following print
sys.stdout.flush()
print("\n\nThe solution contains the following paths: ")
for path, var in pricer.data['vars'].items():
    if sol[var] > 0.1:
        print(f"{sol[var]} * {var}: {path}")

MODEL BUILDER: The number of vehicles is 4
PRICER_PY: The maximal path length is 32
PRICER_C: Graph data successfully copied to C.
PY PRICING: Found 189 paths with reduced costpresolving:
presolving (1 rounds: 1 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 21 variables (0 bin, 0 int, 0 impl, 21 cont) and 22 constraints
     22 constraints of type <linear>
Presolving Time: 0.00


 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
  0.1s|     1 |     0 |    10 |     - |  1233k |   0 | 210 |  22 |  22 |   0 |  0 |   0 |   0 |      --      |      --      |    Inf | unknown
PY PRICING: Found 167 paths with reduced cost
PY PRICING: Found 140 paths with reduced cost
PY PRICING: Found 252 paths with reduced cost
PY PRICING: Found 362 p